In [176]:
import os

import pandas as pd
from dotenv import load_dotenv
import sqlalchemy
from sqlalchemy import create_engine, text
import re
import numpy as np
from sklearn import preprocessing

In [177]:
# load environment variables
load_dotenv(dotenv_path='../.env')

# connection parameters
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PASS')
host = os.environ.get('HOST')
port = os.environ.get('DB_PORT')
database = os.environ.get('DB_NAME')
url = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'

engine = sqlalchemy.create_engine(url)

In [178]:
query = 'select * from races'
race_df = pd.read_sql_query(sql=text(query), con=engine.connect())

In [179]:
read_race_results = 'select * from race_results'
race_results_df = pd.read_sql_query(sql=text(read_race_results), con=engine.connect())

In [180]:
merge_df = pd.merge(race_df, race_results_df, on='id', how='left')
merge_df = merge_df.dropna(subset=["id"])

In [181]:
use_columns = [
    "id", "race_name", "race_place", "number_of_entries", "race_state", "date",
    "box", "horse_order", "sex_and_age", "burden_weight",
    "jockey", "horse_weight", "horse_trainer", "horse_owner", "prize"
]

use_df = merge_df[use_columns]

In [182]:
def get_race_state_features(df):
    return_df = df.copy()
    return_df["race_course"] = df["race_state"].str[1]
    return_df["race_distance"] = df["race_state"].str[2:6]
    return_df["race_weather"] = df["race_state"].str[15]
    return_df["race_state"] = df["race_state"].str[23]
    return_df["race_start"] =  df["race_state"].str[32:37].str.replace(":", "")
    return return_df

In [183]:
def get_sex_and_age(df):
    return_df = df.copy()
    return_df["sex"] = return_df["sex_and_age"].str[0]
    return_df["age"] = return_df["sex_and_age"].str[1]
    return return_df

In [184]:
def get_horse_weight(df):
    return_df = df.copy()
    return_df["difference_weight"] = return_df["horse_weight"].str[3:]
    return_df["difference_weight"] = return_df["difference_weight"].replace(re.compile("\(|\)"), "", regex=True)
    return_df[return_df['difference_weight'] == ''] = -9999
    return_df["difference_weight"] = return_df["difference_weight"].astype(int)
    return_df[return_df['difference_weight'] == -9999] = None
    return_df["horse_weight"] = return_df["horse_weight"].str[0:3]
    return return_df

In [185]:
def get_date(df):
    return_df = df.copy()
    return_df['date'] = return_df['date'].str.split(' ', expand=True)[0]
    return_df['date'] = pd.to_datetime(return_df['date'], format='%Y年%m月%d日').dt.day_of_year
    return_df['date_cos'] = np.cos(2 * np.pi * return_df['date'] / return_df['date'].max())
    return_df['date_sin'] = np.sin(2 * np.pi * return_df['date'] / return_df['date'].max())
    return return_df

In [186]:
def get_all_feature(df):
    return_df = df.copy()
    return_df = get_race_state_features(return_df)
    return_df = get_sex_and_age(return_df)
    return_df = get_horse_weight(return_df)
    return_df = get_date(return_df)
    return return_df

In [187]:
def label_encoder(df, cols):
    return_df = df.copy()
    for col in cols:
        le = preprocessing.LabelEncoder()
        return_df[col] = pd.Series(le.fit_transform(return_df[col]))
    return return_df

In [188]:
encoding_columns = [
    "race_name", "race_place",
    "race_state", "race_course", "race_weather",
    "sex_and_age", "sex",
    "jockey", "horse_trainer", "horse_owner"
]

df = get_all_feature(use_df)
label_encoder(df, encoding_columns).T

,0,1,2,3,4,5,6,7,8,9,...,62047,62048,62049,62050,62051,62052,62053,62054,62055,62056
id,201705010105,201705010105,201705010105,201705010105,201705010105,201705010105,201705010105,201705010105,201705010105,201705010105,...,202307011208,202307011208,202307011211,202307011211,202307011211,202307011211,202307011211,202307011211,202307011211,202307011211
race_name,11,11,11,11,11,11,11,11,11,11,...,13,13,566,566,566,566,566,566,566,566
race_place,3,3,3,3,3,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
number_of_entries,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,...,10.0,10.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
race_state,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
date,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,...,36.0,36.0,36.0,36.0,36.0,36.0,36.0,36.0,36.0,36.0
box,4,4,5,7,5,7,1,1,6,3,...,5,8,2,7,3,5,4,1,6,8
horse_order,8,7,9,13,10,14,2,1,11,6,...,5,9,2,7,3,5,4,1,6,8
sex_and_age,11,11,11,11,11,11,11,11,11,11,...,13,13,21,21,21,21,11,21,21,21
burden_weight,54,54,54,54,54,54,54,54,54,54,...,56,56,56,56,56,56,54,56,56,56
